In [1]:
from dolfinx import geometry, fem, mesh, plot, io
from mpi4py import MPI
from petsc4py import PETSc
from petsc4py.PETSc import ScalarType
from ufl import (TrialFunction, Measure, TestFunction, dx, ds, grad, div, Dx, inner, lhs, rhs)

import timeit
import numpy as np
import pandas as pd

from utils.dolfinx import BoundaryCondition, generate_boundary_measure, eval_pointvalues, project
from meshes import pipe_0_2d, pipe_0_3d
# from utils.plotting import Mpl2DPlotter, Mpl2DAnimator

from IPython.display import Video
from pathlib import Path

In [2]:
save_dir = "/root/Meshes/2d-test-model"
animator_dir = f'{save_dir}/animator'
Path(save_dir).mkdir(parents=True, exist_ok=True)
Path(animator_dir).mkdir(parents=True, exist_ok=True)

U_PATH = f"{save_dir}/uj.xdmf"
UD_PATH = f"{save_dir}/ujd.xdmf"
PRESSURE_PATH = f"{save_dir}/pressure.xdmf"

In [3]:
def problem_setup(N, f, T, t, ct, dt, fluid, bubble_lvl, solver, pc, prefix):    
    """
    Performs problem configuration w.r.t. given parameters
    """
    
    # Mesh and function space definition
    if bubble_lvl > 0.0:
        domain, mt, ft = pipe_0_2d.build_bubble_mesh(N, 0.25, MPI.COMM_WORLD, 0)
    else:
        domain, mt, ft = pipe_0_3d.build_normal_mesh(N, MPI.COMM_WORLD, 0)
        # domain, mt, ft = pipe_0_2d.build_normal_mesh(N, MPI.COMM_WORLD, 0)

    V = fem.VectorFunctionSpace(domain, ("CG", 2))

    u = TrialFunction(V)
    v = TestFunction(V)
    
    # Definition functions of physical characteristics
    fluids = pd.read_csv('../data/physical_properties.csv', sep=';', index_col='Fluid')
    
    ro, c, eta = fluids.loc[fluid, ['Density', 'Speed of sound', 'Viscosity']]
    ro_a, c_a, eta_a = fluids.loc['Air', ['Density', 'Speed of sound', 'Viscosity']]
        
    # Construction of problem form
    GAMMA, BETA = 0.5, 0.5
    
    dx = Measure('dx', subdomain_data=mt, domain=domain)
    
    mm = ScalarType(ro) * inner(u, v) * dx(1)
    aa = ScalarType(ro * c**2) * inner(grad(u), grad(v)) * dx(1)
    cc = ScalarType(4./3 * eta) * inner(grad(u), grad(v)) * dx(1)

    if bubble_lvl > 0.0:
        mm += ScalarType(ro_a) * inner(u, v) * dx(2)
        aa += ScalarType(ro_a * c_a**2) * inner(grad(u), grad(v)) * dx(2)
        cc += ScalarType(4./3 * eta_a) * inner(grad(u), grad(v)) * dx(2)
    
    F = mm + (dt * GAMMA * cc) + (0.5 * dt**2 * BETA * aa)
    
    # Definition of boundary conditions
    measure = generate_boundary_measure([], domain, ft)
    
    u_D = lambda x: [x[0] * 0.0, x[1] * 0.0, x[2] * 0.0]
    u_N1 = fem.Constant(domain, ScalarType((f, 0, 0)))
    u_N2 = fem.Constant(domain, ScalarType((0, 0, 0)))

    bcs = [BoundaryCondition("Dirichlet", 1, u_D, V, u, v, measure).bc,
           BoundaryCondition("Dirichlet", 2, u_D, V, u, v, measure).bc]
     
    nbcs = [#BoundaryCondition("Neumann", 3, u_N2, V, u, v, measure).bc,
            BoundaryCondition("Neumann", 4, u_N1, V, u, v, measure).bc,
            BoundaryCondition("Neumann", 4, u_N2, V, u, v, measure).bc] 
    
    return {
        'Params': (N, fluid, ro, c, f, T, dx, dt, t, ct, GAMMA, BETA, solver, pc, prefix),
        'FunctionSpace': (domain, V, u, v),
        'Forms': (mm, aa, cc), 
        'Problem': (F, bcs, nbcs)
    }

In [4]:
def solve_problem(config):
    N, fluid, ro, c, f, T, dx, dt, t_stop, ct, GAMMA, BETA, solver_type, pc, prefix = config['Params']
    domain, V, u, v = config['FunctionSpace']
    mm, aa, cc = config['Forms']
    F, bcs, nbcs = config['Problem']
    
    if prefix:
        PREFIX = f'{solver_type}_{pc}_{MPI.COMM_WORLD.size}_'
    else:
        PREFIX = ''
    
    U_PATH = f"{save_dir}/{PREFIX}uj.xdmf"
    UD_PATH = f"{save_dir}/{PREFIX}uj_d.xdmf"
    PRESSURE_PATH = f"{save_dir}/{PREFIX}pressure.xdmf"

    xdmf_u = io.XDMFFile(domain.comm, U_PATH, "w")
    xdmf_ud = io.XDMFFile(domain.comm, UD_PATH, "w")
    xdmf_p = io.XDMFFile(domain.comm, PRESSURE_PATH, "w")
    xdmf_u.write_mesh(domain)
    xdmf_ud.write_mesh(domain)
    xdmf_p.write_mesh(domain)
    
    # Create initial condition
    initial_condition = lambda x: [x[0] * 0.0, x[1] * 0.0, x[2] * 0.0]
    
    uj = fem.Function(V)
    uj_d = fem.Function(V)
    uj_dd = fem.Function(V)
    uj_d_d_tg = fem.Function(V)
    
    uj.interpolate(initial_condition)
    uj_d.interpolate(initial_condition)
    uj_dd.interpolate(initial_condition)
    uj_d_d_tg.interpolate(initial_condition)
    
    f1 = fem.Constant(domain, ScalarType((f, 0, 0)))
    
    p = project(ro * c**2 * (Dx(uj[0], 0) + Dx(uj[1], 1)), domain, ("CG", 2))
    
    # Construct the right hand side of the problem
    bilinear_form = fem.form(lhs(F))
    
    A = fem.petsc.assemble_matrix(bilinear_form, bcs=bcs)
    A.assemble()
    
    solver = PETSc.KSP().create(domain.comm)
    solver.setOperators(A)
    solver.setType(solver_type.lower())
    solver.getPC().setType(pc.lower())

    uj_d_d_tg.x.array[:] = dt * GAMMA * uj_d.x.array
        
    # L1 = nbcs[0] + inner(f1, v) * dx(1) + (cc * uj_d) + (aa * uj) + (aa * uj_d_d_tg)
    L1 = nbcs[0] + (cc * uj_d) + (aa * uj) + (aa * uj_d_d_tg)
    L2 = nbcs[1] + (cc * uj_d) + (aa * uj) + (aa * uj_d_d_tg)
    linear_form_1 = fem.form(rhs(L1))
    linear_form_2 = fem.form(rhs(L2))
    
    b1 = fem.petsc.create_vector(linear_form_1)
    b2 = fem.petsc.create_vector(linear_form_2)

#     has_norm = False
#     L2_n, H1_n = 0.0, 0.0

    # Solve problem at each time step
    num_steps = int(T / dt)
    save_step = 1 if num_steps // 200 < 1 else num_steps // 200
    t = 0.0

    for i in range(num_steps):
        print(f'Step:{i}')
        if i == 25:
            break
        t += dt

        # Multiply values of uj_d by dt and GAMMA, 
        # because form does not support muliplication by constant
        uj_d_d_tg.x.array[:] = uj_d.x.array * dt * GAMMA
  
        # b = b1 if int(t / t_stop) % 2 == 0 else b2
        b = b1 if t < t_stop else b2
        linear_form = linear_form_1 if t < t_stop else linear_form_2

        with b.localForm() as loc_b:
            loc_b.set(0)
        fem.petsc.assemble_vector(b, linear_form)

        # Apply Dirichlet boundary condition to the vector
        fem.petsc.apply_lifting(b, [bilinear_form], [bcs])
        b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
        fem.petsc.set_bc(b, bcs)

        # Solve linear problem
        solver.solve(b, uj_dd.vector)
        uj_dd.x.scatter_forward()

        # Update solution
        uj1_d_array = uj_d.x.array + dt * uj_dd.x.array

        uj.x.array[:] = uj.x.array + 0.5 * dt * (uj_d.x.array + uj1_d_array)
        uj_d.x.array[:] = uj1_d_array

        # Save results into the files
        if i % save_step == 0:            
            p = project(ro * c**2 * (Dx(uj[0], 0) + Dx(uj[1], 1)), domain, ("CG", 2))
            
            xdmf_u.write_function(uj, t)
            xdmf_ud.write_function(uj_d, t)
            xdmf_p.write_function(p, t)
        
#         if not has_norm and t >= ct:
#             L2_n = L2_norm(uj)
#             H1_n = H1_norm(uj)

#             has_norm = True
        
    xdmf_u.close()
    xdmf_ud.close()
    xdmf_p.close()
    
#     return L2_n, H1_n

In [ ]:
config = problem_setup(
    N=20,
    f=-1.0e3,
    T=1.7e-2,
    t=1.75e-3,
    ct=2.0e-3,
    dt=8.75e-6,
    fluid='Water',
    bubble_lvl=0.0,
    solver='preonly',
    pc='lu',
    prefix=False
)

solve_problem(config)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 40%] Meshing curve 2 (Line)
Info    : [ 70%] Meshing curve 3 (Circle)
Info    : Done meshing 1D (Wall 0.0289753s, CPU 0.004341s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Cylinder, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.353759s, CPU 0.247635s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 3932 nodes...
Info    : Done tetrahedrizing 3940 nodes (Wall 0.0797597s, CPU 0.07107s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    : Done reconstructing mesh (Wall 0.245395s, CPU 0.227434s)
Info    : Found volume 1
Info    : It. 0 - 0 nodes created - worst tet radius 5.17726 (nodes removed 0 0)
Info    : It. 500 

In [ ]:
# from ufl import Dx

In [ ]:
# domain, mt, ft = pipe_0_2d.build_normal_mesh(20, MPI.COMM_WORLD, 0)

# V = fem.VectorFunctionSpace(domain, ("CG", 2))

In [ ]:
# initial_condition = lambda x: [x[0] * 0.0, x[1] * 0.0, x[2] * 0.0]
# uj = fem.Function(V)
# uj.interpolate(initial_condition)

In [ ]:
# p = 1000 * 1500**2 * (Dx(uj[0], 0) + Dx(uj[1], 1))#div(uj)

# P = fem.Function(V)
# P_expr = fem.Expression(p, V.element.interpolation_points())
# # P.interpolate(P_expr)

In [ ]:
# P_expr.dtype

In [ ]:
# der = fem.Expression([Dx(uj, 0), Dx(uj, 0), Dx(uj, 0)], V.element.interpolation_points())

In [ ]:
# Vs = fem.FunctionSpace(domain, ("Lagrange", 2))
# magnitude = fem.Function(Vs)

In [ ]:
# us = fem.Expression(p, Vs.element.interpolation_points())
# magnitude.interpolate(us)